In [1]:
import pandas as pd
import numpy as np
from sklearn import tree

In [2]:
#botdata = pd.read_csv('botAccounts.csv')
#gooddata= pd.read_csv('GoodAccounts.csv', encoding = 'ISO-8859-1')

In [2]:
testbot = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/bots_data.csv", encoding = 'ISO-8859-1')
testgood = pd.read_csv("/Users/lilixu/Desktop/ML/Machine-Learning-Project/data/nonbots_data.csv", encoding = 'ISO-8859-1')

In [3]:
df = testbot.append(testgood)

In [146]:
testbot.shape

(1056, 20)

In [147]:
testgood.shape

(1176, 20)

In [143]:
df.shape

(2232, 20)

In [46]:
#modify the column name to fit
def adjust_column_name(df):
    for x in df.columns.values:
        df = df.rename(columns={x: x.lower()})
    df = df.rename(columns={'listed_count' : 'listedcount'})

In [6]:
# split the dataset to training & testing
from sklearn.cross_validation import train_test_split
train, test = train_test_split(df, test_size = 0.2)

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [190]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import cross_val_score
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, GaussianNB
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score

In [ ]:
df = testbot.append(testgood).fillna("")

In [196]:
def nb_dt(nb_column_name):
    train, test = train_test_split(df, test_size = 0.2)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count','nb_result']
    count_vect = CountVectorizer()
    
    def convert(column):
        X_word_counts = count_vect.fit_transform(column)
        count_vect.vocabulary_.get(u'algorithm')
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    def convert2(column):
        X_word_counts = count_vect.transform(test[nb_column_name])
        tf_transformer = TfidfTransformer(use_idf=False).fit(X_word_counts)
        X_word_tf = tf_transformer.transform(X_word_counts)
        return X_word_tf.todense()
    
    #Train
    clf = tree.DecisionTreeClassifier(random_state=0)
    M_clf = MultinomialNB().fit(convert(train[nb_column_name]), train['bot'])
    train['nb_result'] = M_clf.predict(convert(train[nb_column_name]))
    clf.fit(train[numfeatures],train["bot"])
    #Test
    test['nb_result'] = M_clf.predict(convert2(train[nb_column_name]))
    
    #Evaluate
    accuracy = accuracy_score(clf.predict(test[numfeatures]),test['bot'])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [accuracy, precison, recall, f1, roc]

In [197]:
for _ in range(20):
    print(nb_dt('description'))

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/ipykernel/__main__.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[0.86353467561521258, 0.88837209302325582, 0.83771929824561409, 0.86230248306997737, 0.86406512857486173]
[0.84340044742729303, 0.81105990783410142, 0.85853658536585364, 0.83412322274881523, 0.84455754888127388]
[0.85011185682326618, 0.82488479262672809, 0.86057692307692313, 0.84235294117647064, 0.85079055358867084]
[0.84787472035794187, 0.86473429951690817, 0.81735159817351599, 0.84037558685446012, 0.8472722903148282]
[0.82774049217002232, 0.82407407407407407, 0.82027649769585254, 0.8221709006928406, 0.82752955319575239]
[0.88590604026845643, 0.87142857142857144, 0.88405797101449279, 0.8776978417266188, 0.88577898550724621]
[0.84787472035794187, 0.87155963302752293, 0.82608695652173914, 0.84821428571428581, 0.84852734922861162]
[0.8523489932885906, 0.80995475113122173, 0.88177339901477836, 0.84433962264150952, 0.85482112573689739]
[0.85458612975391501, 0.875, 0.84120171673819744, 0.85776805251641131, 0.85518029762143521]
[0.83445190156599558, 0.88738738738738743, 0.80081300813008127, 

In [158]:
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

In [198]:
def dt():
    df = testbot.append(testgood)
    numfeatures = ['followers_count', 'friends_count', 'listedcount','favourites_count','statuses_count']
    train, test = train_test_split(df, test_size = 0.2)
    clf.fit(train[numfeatures],train["bot"])
    accuracy = accuracy_score(clf.predict(test[numfeatures]),test['bot'])
    precison = precision_score(clf.predict(test[numfeatures]),test['bot'])
    recall = recall_score(clf.predict(test[numfeatures]),test['bot'])
    f1 = f1_score(clf.predict(test[numfeatures]),test['bot'])
    roc = roc_auc_score(clf.predict(test[numfeatures]),test['bot'])
    return [accuracy, precison, recall, f1, roc]

In [199]:
for _ in range(20):
    print(dt())

[0.80089485458612975, 0.81603773584905659, 0.77578475336322872, 0.79540229885057467, 0.80083880525304296]
[0.81431767337807603, 0.80303030303030298, 0.78325123152709364, 0.79301745635910215, 0.81170758297666157]
[0.82102908277404918, 0.79357798165137616, 0.83173076923076927, 0.81220657276995323, 0.82172312520115864]
[0.86353467561521258, 0.86729857819905209, 0.84722222222222221, 0.85714285714285698, 0.86300505050505061]
[0.83892617449664431, 0.81773399014778325, 0.82587064676616917, 0.82178217821782173, 0.83773207135056427]
[0.87472035794183445, 0.8666666666666667, 0.84924623115577891, 0.85786802030456855, 0.87220376073917982]
[0.83668903803131989, 0.80597014925373134, 0.82653061224489799, 0.81612090680100768, 0.83557606309456045]
[0.8523489932885906, 0.83944954128440363, 0.85514018691588789, 0.84722222222222221, 0.85246279732060481]
[0.85011185682326618, 0.84615384615384615, 0.81683168316831678, 0.83123425692695208, 0.8471913517882399]
[0.86577181208053688, 0.83574879227053145, 0.8693